happy 및 sad 데이터셋이 있다. 

총 80 객의 images 에서, 40개의 happy, 40 개의 sad가 있다.

training accuracy 가 0.999보다 크면, 자동으로 멈추도록, convolutional neural network 을 만드시오. 

Hint : 3개의 convolutional layers 사용

참고사이트: 텐서플로우 놀이터
https://playground.tensorflow.org/

www.streamlit.io

In [1]:
import tensorflow as tf
import os
import zipfile

In [2]:
!wget --no-check-certificate "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" -O "/tmp/happy-or-sad.zip"

--2021-02-26 00:35:12--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 108.177.125.128, 74.125.23.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.03s   

2021-02-26 00:35:12 (95.2 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [ ]:
!unzip /tmp/happy-or-sad.zip -d /tmp/happy-sad

In [4]:
import os

In [5]:
myPATH = '/tmp/happy-sad'

In [6]:
train_happy_dir = os.path.join(myPATH+'/happy')
train_sad_dir = os.path.join(myPATH+'/sad')

In [7]:
# 특정경로의 디렉토리의 파이이름 리스트 가져오기
train_happy_fileList = os.listdir(train_happy_dir)
train_sad_fileList = os.listdir(train_sad_dir)

In [ ]:
## 콜백 함수를 만든다.

[참고사이트](https://streamlit.io/gallery)

In [8]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        i = 0.999
        if (logs.get('accuracy') > i ):
           print("\n정확도 {}가 되어서 학습을 종료합니다".format(i*100))  
           self.model.stop_training = True
        # logs.get('loss')로 하게 되면 lost 결과로 계산할 수 있음
  #객체 생성
myCallback = MyCallback()

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
## 모델링을 하고, 컴파일 코드를 작성한다.

In [10]:
model = Sequential()
model.add (Conv2D (64, (3, 3), activation='relu', input_shape= (150, 150, 3) ))
model.add (MaxPooling2D(2, 2) )
model.add (Conv2D (32, (3, 3), activation='relu')) 
model.add (MaxPooling2D(2, 2) )
model.add (Conv2D (32, (3, 3), activation='relu')) 
model.add (MaxPooling2D(2, 2) )
model.add (Flatten() )
model.add (Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import RMSprop

# learning mate는 천천히 천천히 내려오게 된다 (gradient descent)
# RMSprop(lr= ) lr은 learning mate 0.03 크면  빨리 내려오고 , 적으면 천천히 내려온다
# 
#Regularization rate: overfitting을 막는 놈 참고로만 알고 있자 일단, 
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy'] )

직접 구글 드라이브에 파일을 업로드된 파일에서 오른쪽 마우스 버튼으로 get link를 해서 공유를 해주면 주소를 얻을 수 있다

만약 안되면 구글 드라이브를 직접 마운트해서 사용한다

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1 / 255.0)

train_generator = train_datagen.flow_from_directory(
        myPATH, 
        
        # 타켓 사이즈는 여기서 바꿀 수 있음 : 여기서 바꾸면 input layer의 input_shape도 같게 해줘야 함
        target_size= (150, 150),
        class_mode='binary',
        #batch_size 생략 시 32로 됨
        batch_size = 8
)

# 자동으로 다음처럼 출력된다. : 'Found 80 images belonging to 2 classes'

Found 10 images belonging to 2 classes.


따로 업로드한 파일을 사용할 경우 ImageDataGenerator  
학습시 validation_data= test_generator 로 
사용하기 위해서

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1 / 255.0)

        # 또는 직접 content 디렉토리에 업로드해도 됨~ (단, 이경우는 세션?이 종료되면 파일도 지워진다)
        # 새로운 이미지 업로드 후 사용할 경우에 
myNewPath = '/content/test_set/'

test_generator = train_datagen.flow_from_directory(
               
        myNewPath,
        
        # 타켓 사이즈는 여기서 바꿀 수 있음 : 여기서 바꾸면 input layer의 input_shape도 같게 해줘야 함
        target_size= (150, 150),
        class_mode='binary',
        #batch_size 생략 시 32로 됨
        batch_size = 8
)

# 자동으로 다음처럼 출력된다. : 'Found 80 images belonging to 2 classes'

In [35]:
# 해당 디렉토리로 happy가 0인지 확인
test_generator.class_indices

{'happy': 0, 'sad': 1}

In [37]:
# 학습한다. 에포크도 지정하시오.

history = model.fit(train_generator, batch_size=40, epochs=30, verbose=1, callbacks=[ myCallback ], validation_data= test_generator)

Epoch 1/30
2/2 [==============================] - 1s 371ms/step - loss: 0.7332 - accuracy: 0.3000 - val_loss: 0.6796 - val_accuracy: 0.4000
Epoch 2/30
2/2 [==============================] - 1s 336ms/step - loss: 0.6793 - accuracy: 0.4000 - val_loss: 0.5998 - val_accuracy: 0.6000
Epoch 3/30
2/2 [==============================] - 1s 336ms/step - loss: 0.5949 - accuracy: 0.6000 - val_loss: 0.6418 - val_accuracy: 0.5000
Epoch 4/30
2/2 [==============================] - 1s 320ms/step - loss: 0.6113 - accuracy: 0.6000 - val_loss: 0.4752 - val_accuracy: 0.7000
Epoch 5/30
2/2 [==============================] - 1s 315ms/step - loss: 0.5416 - accuracy: 0.7000 - val_loss: 0.4849 - val_accuracy: 0.8000
Epoch 6/30
2/2 [==============================] - 1s 309ms/step - loss: 0.4684 - accuracy: 0.8000 - val_loss: 0.3562 - val_accuracy: 0.9000
Epoch 7/30
2/2 [==============================] - 1s 523ms/step - loss: 0.4542 - accuracy: 0.6000 - val_loss: 0.3120 - val_accuracy: 1.0000
Epoch 8/30
2/2 [====

In [ ]:
#학습데이터 정확도를 너무 믿지 마라. overfitting 되었을 수 있다

인터넷으로 사진 받아서 예측해보기

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  # 디렉토리안에 있는 하위디렉토리의 알파벳 순으로 순서가 됨 먼저가 (예: happy가 0, sad는 1)
  # 픽사베이등에서 받은 파일들을 처리할 때
  # 이미지를 load_img() 통해 target_size를 300x300 으로 바꿔준다 
  img = image.load_img(path, target_size=(150, 150) )
  # image를 ndarray로 
  x = image.img_to_array(img)

  # 원래 3차원 이지
  print(x.shape)

  # 행으로 한 차원을 늘림 (150, 150, 1) 이 되게 됨
  x = np.expand_dims(x, axis = 0)
  print(x.shape)

  images = np.vstack([x])
  classes = model.predict( images, batch_size = 10)

  print(classes)

  if classes[0] > 0.5 :
    print(fn + " is so sad")
  else:
    print(fn + " is so happy")


Saving happy17.png to happy17 (2).png
Saving happy16.png to happy16 (2).png
Saving happy15.png to happy15 (2).png
Saving happy14.png to happy14 (2).png
Saving happy13.png to happy13 (2).png
Saving happy12.png to happy12 (2).png
Saving happy11.png to happy11 (2).png
(150, 150, 3)
(1, 150, 150, 3)
[[1.]]
happy17.png is so sad
(150, 150, 3)
(1, 150, 150, 3)
[[1.]]
happy16.png is so sad
(150, 150, 3)
(1, 150, 150, 3)
[[0.]]
happy15.png is so happy
(150, 150, 3)
(1, 150, 150, 3)
[[1.]]
happy14.png is so sad
(150, 150, 3)
(1, 150, 150, 3)
[[1.]]
happy13.png is so sad
(150, 150, 3)
(1, 150, 150, 3)
[[1.]]
happy12.png is so sad
(150, 150, 3)
(1, 150, 150, 3)
[[1.]]
happy11.png is so sad
